## Imports

In [1]:
import numpy as np
import os
import cv2

from collections import deque
from matplotlib import pyplot as plt
from PIL import ImageOps, ImageEnhance, Image
from ipywidgets import interact
from cytomine import Cytomine
from cytomine.models import *

## Cytomine connection parameters

In [2]:
host = 'https://learn.cytomine.be'
public_key = 'd42f0ab8-6c8a-4f0f-9002-dc1c8406b196'
private_key = 'db6649ac-7463-4032-987a-eafb5f224ffd'

## Downloading the images

In [12]:
only_our_project = True
project_id_group05 = 9874553 # This is the CV2021_GROUP05 IDs

with Cytomine(host=host, public_key=public_key, private_key=private_key) as cytomine:

    if only_our_project:
        # Retrieve only the images we annotated ourselves
        image_instances = ImageInstanceCollection().fetch_with_filter("project", project_id_group05)
        for image in image_instances:
            path = os.path.join('../Images/Our_images/', str(project_id_group05), image.originalFilename)
            image.download(path, override=False)
    else:
        # Retrieve all annotated images
        projects = ProjectCollection().fetch()
        for project in projects:
            image_instances = ImageInstanceCollection().fetch_with_filter("project", project.id)
            for image in image_instances:
                path = os.path.join('../Images/All_images/', str(project.id), image.originalFilename)
                image.download(path, override=False)

[2021-10-13 16:12:11,117][INFO] [GET] [currentuser] CURRENT USER - 9873406 : s162782 | 200 OK
[2021-10-13 16:12:11,139][INFO] [GET] [imageinstance collection] 1 objects | 200 OK
[2021-10-13 16:12:39,563][INFO] File downloaded successfully from https://learn.cytomine.be/api/imageinstance/11208725/download with parameters {}


## Fetching sequence

In [4]:
video_path = 'dataset/raw/CV2021_GROUP05/group5.mp4'

## Background subtraction : single reference frame method

In [5]:
video = cv2.VideoCapture(video_path)

# Simplest method : choose the first frame as background
ret, img = video.read()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# print(img.shape)
# print(gray.shape)
background = gray.copy()
threshold = 25

while video.isOpened():
    
    ret, img = video.read()
    
    if not ret:
        video.release()
        cv2.destroyAllWindows()
        break
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = np.abs(gray-background) > threshold
    masked_gray = gray.copy()
    np.putmask(masked_gray, np.logical_not(mask), 0.)
    
    # Displaying original sequence, mask and masked sequence
    cv2.imshow('Sequence', gray)
    cv2.imshow('Mask', mask.astype(float))
    cv2.imshow('Masked sequence', masked_gray)

    k = cv2.waitKey(0) & 0xff
    if k == ord('q'): # Allows to close window when pressing the 'q' key
        video.release()
        cv2.destroyAllWindows()
        break
    



error: OpenCV(4.5.4) /tmp/pip-req-build-9vck9bv0/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


## Defining filters

In [5]:
def average_filter(kx, ky, img):
    kernel = np.ones((ky, kx)) / (kx*ky)
    avg_image = cv2.filter2D(img, -1, kernel)

    return avg_image

In [6]:
def gaussian_filter(kx, ky, img):
    kernel = (ky, kx)
    gauss_image = cv2.GaussianBlur(img, kernel, 0)
    
    return gauss_image

## Background subtraction : exponential update of reference frame method

In [8]:
video = cv2.VideoCapture(video_path)

# Extracting first frame
ret, img = video.read()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# print(img.shape)
# print(gray.shape)
background = gray.copy()
background = gaussian_filter(5,5,background)
threshold = 25
alpha = 0.05

while video.isOpened():
    
    ret, img = video.read()
    
    if not ret:
        video.release()
        cv2.destroyAllWindows()
        break
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gaussian_filter(5,5,gray)
    mask = np.abs(gray-background) > threshold
    masked_gray = gray.copy()
    np.putmask(masked_gray, np.logical_not(mask), 0.)
    
    # Displaying original sequence, mask and masked sequence
    cv2.imshow('Sequence', gray)
    cv2.imshow('Mask', mask.astype(float))
    cv2.imshow('Masked sequence', masked_gray)
    
    background = alpha * gray + (1-alpha) * background

    k = cv2.waitKey(0) & 0xff
    if k == ord('q'): # Allows to close window when pressing the 'q' key
        video.release()
        cv2.destroyAllWindows()
        break

## Background subtraction : Median of last N frames

In [3]:
video = cv2.VideoCapture(video_path)

# Extracting first frame
ret, img = video.read()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# print(img.shape)
# print(gray.shape)
last_N_frames = deque()
last_N_frames.append(gray)
threshold = 25

N = 10
i = 0

while video.isOpened():
    
    ret, img = video.read()
    
    if not ret:
        video.release()
        cv2.destroyAllWindows()
        break
    
    background = np.median([frame for frame in last_N_frames], axis=0)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gaussian_filter(5,5,gray)
    mask = np.abs(gray-background) > threshold
    masked_gray = gray.copy()
    np.putmask(masked_gray, np.logical_not(mask), 0.)
    
#     masked_gray = average_filter(3,3,masked_gray)
    
    # Displaying original sequence, mask and masked sequence
    cv2.imshow('Sequence', gray)
    cv2.imshow('Mask', mask.astype(float))
    cv2.imshow('Masked sequence', masked_gray)
    
    if i >= N:
        last_N_frames.popleft()
    last_N_frames.append(gray)
    i += 1

    k = cv2.waitKey(0) & 0xff
    if k == ord('q'): # Allows to close window when pressing the 'q' key
        video.release()
        cv2.destroyAllWindows()
        break

NameError: name 'gaussian_filter' is not defined

In [ ]:
def display_video(path):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print("Error while opening file! \n")

    prev = 0
    while cap.isOpened():
        time_elapsed = time.time() - prev
        if time_elapsed <= 1. / FRAME_RATE:
            continue

        prev = time.time()
        ret, frame = cap.read()
        if ret:
            cv2.imshow("Frame", frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()
